In [131]:
import mysql.connector
import pandas as pd
import re
import numpy as np

In [304]:
con=mysql.connector.connect(
    host="localhost",
    user="root",
    password="DDDwwwkkk77::work",
    database="world_layoffs"
)

In [305]:
query='''Select * From layoffs'''

In [306]:
df=pd.read_sql_query(query,con)

C:\Users\Nada\AppData\Local\Temp\ipykernel_11344\984522191.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query,con)


In [307]:
df

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
0,Atlassian,Sydney,Other,500.0,0.05,3/6/2023,Post-IPO,Australia,210.0
1,SiriusXM,New York City,Media,475.0,0.08,3/6/2023,Post-IPO,United States,525.0
2,Alerzo,Ibadan,Retail,400.0,None,3/6/2023,Series B,Nigeria,16.0
3,UpGrad,Mumbai,Education,120.0,None,3/6/2023,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,3/3/2023,Unknown,Brazil,788.0
...,...,...,...,...,...,...,...,...,...
2356,Blackbaud,Charleston,Other,500.0,0.14,None,Post-IPO,United States,NaN
2357,Yahoo,SF Bay Area,Consumer,1600.0,0.2,2/9/2023,Acquired,United States,6.0
2358,Hibob,Tel Aviv,HR,70.0,0.3,3/30/2020,Series A,Israel,45.0
2359,Casper,New York City,Retail,NaN,None,9/14/2021,Post-IPO,United States,339.0


In [308]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2361 entries, 0 to 2360
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   company                2361 non-null   object 
 1   location               2361 non-null   object 
 2   industry               2360 non-null   object 
 3   total_laid_off         1621 non-null   float64
 4   percentage_laid_off    1576 non-null   object 
 5   date                   2360 non-null   object 
 6   stage                  2355 non-null   object 
 7   country                2361 non-null   object 
 8   funds_raised_millions  2152 non-null   float64
dtypes: float64(2), object(7)
memory usage: 166.1+ KB


#### Check Duplicated Rows & Drop IT IF ANY.

In [309]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2356    False
2357     True
2358     True
2359     True
2360     True
Length: 2361, dtype: bool

In [310]:
df[df.duplicated()]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1492,Cazoo,London,Transportation,750.0,0.15,6/7/2022,Post-IPO,United Kingdom,2000.0
2357,Yahoo,SF Bay Area,Consumer,1600.0,0.2,2/9/2023,Acquired,United States,6.0
2358,Hibob,Tel Aviv,HR,70.0,0.3,3/30/2020,Series A,Israel,45.0
2359,Casper,New York City,Retail,NaN,None,9/14/2021,Post-IPO,United States,339.0
2360,Wildlife Studios,Sao Paulo,Consumer,300.0,0.2,11/28/2022,Unknown,Brazil,260.0


In [311]:
df[df["company"]=="Yahoo"]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
118,Yahoo,SF Bay Area,Consumer,1600.0,0.2,2/9/2023,Acquired,United States,6.0
2357,Yahoo,SF Bay Area,Consumer,1600.0,0.2,2/9/2023,Acquired,United States,6.0


In [312]:
df.drop_duplicates(inplace=True)

In [314]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2352    False
2353    False
2354    False
2355    False
2356    False
Length: 2356, dtype: bool

In [313]:
df[df["company"]=="Yahoo"]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
118,Yahoo,SF Bay Area,Consumer,1600.0,0.2,2/9/2023,Acquired,United States,6.0


### Standarize Each Column.

#### 1. Company Column.

In [315]:
df["company"].unique()

array(['Atlassian', 'SiriusXM', 'Alerzo', ..., 'Panda Squad',
       'Tamara Mellon', 'Blackbaud'], dtype=object)

#### Checking For Whitespaces.

In [316]:
df[df["company"]==df["company"].str.strip()]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
0,Atlassian,Sydney,Other,500.0,0.05,3/6/2023,Post-IPO,Australia,210.0
1,SiriusXM,New York City,Media,475.0,0.08,3/6/2023,Post-IPO,United States,525.0
2,Alerzo,Ibadan,Retail,400.0,None,3/6/2023,Series B,Nigeria,16.0
3,UpGrad,Mumbai,Education,120.0,None,3/6/2023,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,3/3/2023,Unknown,Brazil,788.0
...,...,...,...,...,...,...,...,...,...
2352,HopSkipDrive,Los Angeles,Transportation,8.0,0.1,3/13/2020,Unknown,United States,45.0
2353,Panda Squad,SF Bay Area,Consumer,6.0,0.75,3/13/2020,Seed,United States,1.0
2354,Tamara Mellon,Los Angeles,Retail,20.0,0.4,3/12/2020,Series C,United States,90.0
2355,EasyPost,Salt Lake City,Logistics,75.0,None,3/11/2020,Series A,United States,12.0


#### Get Rid Of Whitespaces.

In [317]:
df["company"]=df["company"].str.strip()

In [318]:
df[df["company"]!=df["company"].str.strip()]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions


### Location Column.

#### Checking For Whitespaces.

In [319]:
df[df["location"]!=df["location"].str.strip()]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions


In [320]:
df["location"].unique()

array(['Sydney', 'New York City', 'Ibadan', 'Mumbai', 'Sao Paulo',
       'SF Bay Area', 'Seattle', 'Boston', 'Blumenau', 'Pune',
       'Bengaluru', 'Los Angeles', 'Boise', 'Chicago', 'Paris',
       'Baton Rouge', 'Munich', 'Denver', 'Tel Aviv', 'Stockholm',
       'Orlando', 'Ann Arbor', 'Berlin', 'Vancouver', 'Washington D.C.',
       'London', 'San Antonio', 'St. Louis', 'Pittsburgh', 'Tokyo',
       'Lagos', 'Seoul', 'Chennai', 'Shenzen', 'Portland', 'Atlanta',
       'Albany', 'Milan', 'Singapore', 'Jakarta', 'Philadelphia',
       'Columbus', 'Tallinn', 'Phoenix', 'Toronto', 'Melbourne', 'Dublin',
       'Austin', 'New Delhi', 'Manchester', 'Miami', 'Helsinki',
       'Detroit', 'Frankfurt', 'Waterloo', 'Amsterdam', 'Barcelona',
       'Karlsruhe', 'Curitiba', 'Charlotte', 'Las Vegas', 'New Haven',
       'Montreal', 'Coimbra', 'Walldorf', 'Jersey City', 'Reno', 'Kiel',
       'Gurugram', 'Nashville', 'Mexico City', 'Oxford', 'Calgary',
       'Boulder', 'Wilmington', 'Cincinna

In [321]:
df["location"].value_counts()

location
SF Bay Area      613
New York City    248
Boston           109
Los Angeles       93
Bengaluru         81
                ... 
Prague             1
Yangon             1
New Orleans        1
Tampa Bay          1
Charleston         1
Name: count, Length: 191, dtype: int64

### Industry Column.

In [322]:
df["industry"].unique()

array(['Other', 'Media', 'Retail', 'Education', 'Real Estate',
       'Transportation', 'Marketing', '', 'Healthcare', 'Security',
       'Food', 'Fitness', 'Consumer', 'Logistics', 'HR', 'Support',
       'Travel', 'Crypto', 'Finance', 'Data', 'Sales', 'Infrastructure',
       'Hardware', 'Product', 'Construction', 'Legal', 'Energy', None,
       'Manufacturing', 'Recruiting', 'Aerospace', 'Crypto Currency',
       'Fin-Tech', 'CryptoCurrency'], dtype=object)

In [323]:
df.dropna(subset=['industry'], inplace=True)

In [325]:
df[df["industry"]==""]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
8,Airbnb,SF Bay Area,,30.0,None,3/3/2023,Post-IPO,United States,6400.0
736,Juul,SF Bay Area,,400.0,0.3,11/10/2022,Unknown,United States,1500.0
1595,Carvana,Phoenix,,2500.0,0.12,5/10/2022,Post-IPO,United States,1600.0


#### Replace Blanks with Null

In [326]:
df["industry"]=df["industry"].replace("","Nan")

In [327]:
df[df["industry"]=="Nan"]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
8,Airbnb,SF Bay Area,Nan,30.0,None,3/3/2023,Post-IPO,United States,6400.0
736,Juul,SF Bay Area,Nan,400.0,0.3,11/10/2022,Unknown,United States,1500.0
1595,Carvana,Phoenix,Nan,2500.0,0.12,5/10/2022,Post-IPO,United States,1600.0


In [324]:
df[df["industry"].str.contains('Crypto',na=False,regex=True)]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
26,Protego Trust Bank,Seattle,Crypto,NaN,0.5,3/1/2023,Series A,United States,70.0
52,Dapper Labs,Vancouver,Crypto,NaN,0.2,2/23/2023,Series D,United States,607.0
53,Messari,New York City,Crypto,NaN,0.15,2/23/2023,Series B,United States,61.0
58,Immutable,Sydney,Crypto,NaN,0.11,2/22/2023,Series C,Australia,279.0
61,Polygon,Bengaluru,Crypto,100.0,0.2,2/21/2023,Unknown,India,451.0
...,...,...,...,...,...,...,...,...,...
2049,BitGo,SF Bay Area,Crypto,NaN,0.12,4/17/2020,Series B,United States,69.0
2060,Purse,SF Bay Area,Crypto,NaN,1,4/16/2020,Seed,United States,1.0
2109,CipherTrace,SF Bay Area,Crypto,NaN,None,4/9/2020,Unknown,United States,18.0
2110,Elliptic,London,Crypto,NaN,0.3,4/9/2020,Series B,United Kingdom,40.0


In [330]:
df[df["industry"]=="Crypto Currency"]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
901,GSR,Hong Kong,Crypto Currency,NaN,None,10/11/2022,Unknown,Hong Kong,NaN
1272,Unstoppable Domains,SF Bay Area,Crypto Currency,42.0,0.25,7/14/2022,Series B,United States,7.0


In [332]:
df[df["industry"]=="CryptoCurrency"]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1258,Gemini,New York City,CryptoCurrency,68.0,0.07,7/18/2022,Unknown,United States,423.0


In [331]:
df[df["industry"]=="Crypto"]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
26,Protego Trust Bank,Seattle,Crypto,NaN,0.5,3/1/2023,Series A,United States,70.0
52,Dapper Labs,Vancouver,Crypto,NaN,0.2,2/23/2023,Series D,United States,607.0
53,Messari,New York City,Crypto,NaN,0.15,2/23/2023,Series B,United States,61.0
58,Immutable,Sydney,Crypto,NaN,0.11,2/22/2023,Series C,Australia,279.0
61,Polygon,Bengaluru,Crypto,100.0,0.2,2/21/2023,Unknown,India,451.0
...,...,...,...,...,...,...,...,...,...
2049,BitGo,SF Bay Area,Crypto,NaN,0.12,4/17/2020,Series B,United States,69.0
2060,Purse,SF Bay Area,Crypto,NaN,1,4/16/2020,Seed,United States,1.0
2109,CipherTrace,SF Bay Area,Crypto,NaN,None,4/9/2020,Unknown,United States,18.0
2110,Elliptic,London,Crypto,NaN,0.3,4/9/2020,Series B,United Kingdom,40.0


In [333]:
def change_crypto(x):
    if 'Crypto' in x:
        x="Crypto"
    return x

In [334]:
change_crypto("CryptoCurrency")

'Crypto'

In [335]:
df["industry"]=df["industry"].apply(change_crypto)

In [336]:
df[df["industry"]=="CryptoCurrency"]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions


In [337]:
df[df["industry"]=="Crypto Currency"]

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions


### Date Column.

#### Change Column datatype

In [338]:
df["date"]=pd.to_datetime(df["date"])

In [339]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2355 entries, 0 to 2356
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   company                2355 non-null   object        
 1   location               2355 non-null   object        
 2   industry               2355 non-null   object        
 3   total_laid_off         1617 non-null   float64       
 4   percentage_laid_off    1571 non-null   object        
 5   date                   2354 non-null   datetime64[ns]
 6   stage                  2349 non-null   object        
 7   country                2355 non-null   object        
 8   funds_raised_millions  2146 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 184.0+ KB


In [340]:
df["date"]

0      2023-03-06
1      2023-03-06
2      2023-03-06
3      2023-03-06
4      2023-03-03
          ...    
2352   2020-03-13
2353   2020-03-13
2354   2020-03-12
2355   2020-03-11
2356          NaT
Name: date, Length: 2355, dtype: datetime64[ns]

### Country Column.

In [341]:
df["country"].unique()

array(['Australia', 'United States', 'Nigeria', 'India', 'Brazil',
       'United States.', 'France', 'Germany', 'Israel', 'Sweden',
       'United Kingdom', 'Japan', 'South Korea', 'China', 'Italy',
       'Singapore', 'Indonesia', 'Estonia', 'Canada', 'Ireland',
       'Finland', 'Netherlands', 'Spain', 'Portugal', 'Chile', 'Colombia',
       'Argentina', 'Seychelles', 'Austria', 'Mexico', 'Switzerland',
       'Egypt', 'Kenya', 'Luxembourg', 'Greece', 'Poland', 'Norway',
       'Belgium', 'Denmark', 'Hong Kong', 'New Zealand', 'Malaysia',
       'Hungary', 'Vietnam', 'Thailand', 'Lithuania', 'Ghana', 'Senegal',
       'Pakistan', 'United Arab Emirates', 'Peru', 'Bahrain', 'Romania',
       'Turkey', 'Russia', 'Uruguay', 'Bulgaria', 'South Africa',
       'Czech Republic', 'Myanmar'], dtype=object)

In [342]:
df["country"]=df["country"].apply(lambda x:x.split('.')[0])

In [343]:
df["country"].unique()

array(['Australia', 'United States', 'Nigeria', 'India', 'Brazil',
       'France', 'Germany', 'Israel', 'Sweden', 'United Kingdom', 'Japan',
       'South Korea', 'China', 'Italy', 'Singapore', 'Indonesia',
       'Estonia', 'Canada', 'Ireland', 'Finland', 'Netherlands', 'Spain',
       'Portugal', 'Chile', 'Colombia', 'Argentina', 'Seychelles',
       'Austria', 'Mexico', 'Switzerland', 'Egypt', 'Kenya', 'Luxembourg',
       'Greece', 'Poland', 'Norway', 'Belgium', 'Denmark', 'Hong Kong',
       'New Zealand', 'Malaysia', 'Hungary', 'Vietnam', 'Thailand',
       'Lithuania', 'Ghana', 'Senegal', 'Pakistan',
       'United Arab Emirates', 'Peru', 'Bahrain', 'Romania', 'Turkey',
       'Russia', 'Uruguay', 'Bulgaria', 'South Africa', 'Czech Republic',
       'Myanmar'], dtype=object)

In [344]:
df

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
0,Atlassian,Sydney,Other,500.0,0.05,2023-03-06,Post-IPO,Australia,210.0
1,SiriusXM,New York City,Media,475.0,0.08,2023-03-06,Post-IPO,United States,525.0
2,Alerzo,Ibadan,Retail,400.0,None,2023-03-06,Series B,Nigeria,16.0
3,UpGrad,Mumbai,Education,120.0,None,2023-03-06,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,2023-03-03,Unknown,Brazil,788.0
...,...,...,...,...,...,...,...,...,...
2352,HopSkipDrive,Los Angeles,Transportation,8.0,0.1,2020-03-13,Unknown,United States,45.0
2353,Panda Squad,SF Bay Area,Consumer,6.0,0.75,2020-03-13,Seed,United States,1.0
2354,Tamara Mellon,Los Angeles,Retail,20.0,0.4,2020-03-12,Series C,United States,90.0
2355,EasyPost,Salt Lake City,Logistics,75.0,None,2020-03-11,Series A,United States,12.0


In [345]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2355 entries, 0 to 2356
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   company                2355 non-null   object        
 1   location               2355 non-null   object        
 2   industry               2355 non-null   object        
 3   total_laid_off         1617 non-null   float64       
 4   percentage_laid_off    1571 non-null   object        
 5   date                   2354 non-null   datetime64[ns]
 6   stage                  2349 non-null   object        
 7   country                2355 non-null   object        
 8   funds_raised_millions  2146 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 184.0+ KB
